# Digit Recognizer

In [ ]:
from pprint import pprint

## Download the datasets

The data should be downloaded from these links and placed in the `./datasets` folder, named as indicated:

- [train.csv](https://www.kaggle.com/c/digit-recognizer/download/train.csv) | as `digit_train.csv`
- [test.csv](https://www.kaggle.com/c/digit-recognizer/download/test.csv) | as `digit_test.csv`

## Load the datasets

In [ ]:
import csv

def get_batch(filename, n=100):
    """Return a batch of training data."""
    with open(filename, 'r') as csvfile:
        next(csvfile) # Skip header
        datareader = csv.reader(csvfile)
        batch = list()
        count = 0
        for row in datareader:
            if count < n:
                batch.append(row)
                count += 1
            else:
                yield batch
                batch = [row]
                count = 1
    if count != 0:
        yield batch
        
def train_batch(n=100):
    return get_batch('./datasets/digit_train.csv', n)

def test_batch(n=100):
    return get_batch('./datasets/digit_test.csv', n)